In [1]:
import ROOT
import numpy as np
from ROOT import TGraphErrors
from ROOT import TVectorT
import matplotlib.pyplot as plt
from matplotlib import ticker
from root_np_functions import *
from plotting_functions import *

from resolutions import *

Welcome to JupyROOT 6.18/00


In [2]:
from scipy.optimize import curve_fit
def gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def double_gauss(x,a,mu0,sigma0,b,mu1,sigma1):
    return (a * np.exp(-(x - mu0) ** 2 / (2 * sigma0 ** 2)) +
            b * np.exp(-(x - mu1) ** 2 / (2 * sigma1 ** 2)) )

# Compare Charged Truth and Reco

In [4]:
filename = "DeltaR_histograms_reco_NoCuts_output_mom_res_sigma_eta_5_p_6_B_3.0.root"
filename_14T="DeltaR_histograms_reco_NoCuts_output_mom_res_sigma_eta_5_p_6_B_1.4.root"
file = ROOT.TFile.Open(filename)


eta_binning = {}
eta_binning["edges"],N_eta= TVT_to_numpy(file,"TVT_eta_bin")
eta_binning["centers"], eta_binning["widths"] = get_binning_from_edges(eta_binning["edges"])

mom_binning = {}
mom_binning["edges"],N_mom = TVT_to_numpy(file,"TVT_mom_bin")
mom_binning["centers"], mom_binning["widths"] = get_binning_from_edges(mom_binning["edges"])

N_mom = len(mom_binning["centers"])
N_eta = len(eta_binning["centers"])

eta_colors = get_colors(plt.cm.winter, N_eta)
mom_colors = get_colors(plt.cm.autumn, N_mom)


B_Fields = [1.4,3.0]
B_Strings = ["1p4","3"]
N_divs = N_mom
xlims=[[-0.04,0.04]]
prefixes = ["eDelta_dph"] #Types of Resolutions

h1dict = {}
for B_String,B in zip(B_Strings,B_Fields):
        if (B==1.4): 
            file = ROOT.TFile.Open(filename_14T)
        if (B==3.0):
            file = ROOT.TFile.Open(filename)
        for s in prefixes:
            h1dict[s+"edges"],h1dict[s+"centers"],h1dict[s+"widths"] = get_th1_binning_np(file.Get("eD_dph_histos/h1_eDelta_dph_p_et_bins_0_p_0_bin","eD_dph_histos")

            for eta in range(N_eta):
                for p in range(N_mom):
                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    h1dict[identifier+"_vals"],h1dict[identifier+"_errors"] = TH1_to_numpy_wErrors_old(file,"h1_eDelta_dph_p_et_bins_%i_p_%i_bin"%(eta,p),False,False,"eD_dph_histos")
                    h1=file.Get("eD_dph_histos/"+"h1_eDelta_dph_p_et_bins_%i_p_%i_bin"%(eta,p))
                    h1dict[identifier+"_avg"] = h1.GetMean()
                    h1dict[identifier+"_sigma"] = h1.GetStdDev()
                    h1dict[identifier+"_N"] = h1.GetEntries() #For weighted averaging
                    #print(h1.GetMean())
                    
np.save("./np_arrays/indv_histos.npy",h1dict)

TypeError: get_th1_binning_np() takes 1 positional argument but 3 were given

In [ ]:
string_selections = ["../../output/reco_NoCuts_"] #Reco Cuts. First two may not sum to 3rd

for s,xlim in zip(prefixes,xlims):
    for B_String,B in zip(B_Strings,B_Fields):
        fig = plt.figure(figsize=(18,12))
        plt.xticks([])
        plt.yticks([])
        plt.title(r"$\Delta\varphi$ Distributions (B = %1.1f T)"%(B),fontsize=40,y=1.04,verticalalignment="bottom")
        i=0
        for eta in range(N_eta):

            for p in range(N_mom):
                ax = fig.add_subplot(N_eta,N_divs,i+1)
                i+=1
                if eta==0:
                    p_range_string = r"$%1.1f < p < %1.1f$"%(mom_binning["edges"][p],mom_binning["edges"][p+1])
                    plt.text(0.5,1.18,p_range_string,ha="center",va="top",size=20,transform=ax.transAxes)
                if p==0:
                    eta_range_string = r"$%1.1f < \eta < %1.1f$"%(eta_binning["edges"][eta],eta_binning["edges"][eta+1])
                    #plt.text(0.5,1.12,eta_range_string,ha="center",va="top",size=15,alpha=0.7,transform=ax.transAxes)
                    plt.ylabel(eta_range_string,fontsize=16)
                for col in ["r","b","g"]:

                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    plt.errorbar(h1dict[s+"centers"],h1dict[identifier+"_vals"],yerr=h1dict[identifier+"_errors"],fmt=".",color=col,alpha=0.8)
                    
                    #aesthetics
                    plt.xlim(xlim)
                    plt.xticks(fontsize=8)
                    plt.yticks(fontsize=8)
                    mean = h1dict[identifier+"_avg"]
                    #plt.axvline(x=h1dict[identifier+"_avg"],color=col,linestyle="--",alpha=0.5)
                    #stddev = h1dict[identifier+"_sigma"]
                    #ax.axvspan(mean-stddev, mean+stddev, alpha=0.5, color=col)

                    plt.text(0.98,0.95,"B = %1.1f T"%(B),ha="right",va="top",size=15,alpha=0.7,transform=ax.transAxes)
        plt.tight_layout()
        plt.savefig(s+"_"+B_String+"_dDeltaPhi_Overlays.pdf")

# Compare Full and Charged Truth Jets

In [ ]:
filename = "../../output/reco_NoCuts_output_mom_res_3T_combinedsigma_eta_5_p_6_.root"
filename_14T="../../output/reco_NoCuts_output_mom_res_1p4T_combinedsigma_eta_5_p_6_.root"
file = ROOT.TFile.Open(filename)
#file.ls()


eta_binning = {}
eta_binning["edges"],N_eta= TVT_to_numpy(file,"TVT_eta_bin")
eta_binning["centers"], eta_binning["widths"] = get_binning_from_edges(eta_binning["edges"])

mom_binning = {}
mom_binning["edges"],N_mom = TVT_to_numpy(file,"TVT_mom_bin")
mom_binning["centers"], mom_binning["widths"] = get_binning_from_edges(mom_binning["edges"])

N_mom = len(mom_binning["centers"])
N_eta = len(eta_binning["centers"])

eta_colors = get_colors(plt.cm.winter, N_eta)
mom_colors = get_colors(plt.cm.autumn, N_mom)


B_Fields = [1.4,3.0]
B_Strings = ["1p4","3"]
N_divs = N_mom
xlims=[[-0.04,0.04]]
prefixes = ["efulljet_Delta_dph"] #Types of Resolutions

h1dict_full = {}
for B_String,B in zip(B_Strings,B_Fields):
        if (B==1.4): 
            file = ROOT.TFile.Open(filename_14T)
        if (B==3.0):
            file = ROOT.TFile.Open(filename)
        for s in prefixes:
            h1dict_full[s+"edges"],h1dict_full[s+"centers"],h1dict_full[s+"widths"] = get_th1_binning_np(file,"h1_efulljet_Delta_dph_p_et_bins_0_p_0_bin","eD_dph_histos")

            for eta in range(N_eta):
                for p in range(N_mom):
                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    h1dict_full[identifier+"_vals"],h1dict_full[identifier+"_errors"] = TH1_to_numpy_wErrors_old(file,"h1_efulljet_Delta_dph_p_et_bins_%i_p_%i_bin"%(eta,p),False,False,"eD_dph_histos")
                    h1=file.Get("eD_dph_histos/"+"h1_efulljet_Delta_dph_p_et_bins_%i_p_%i_bin"%(eta,p))
                    h1dict_full[identifier+"_avg"] = h1.GetMean()
                    h1dict_full[identifier+"_sigma"] = h1.GetStdDev()
                    #print(h1.GetMean())
                    
np.save("./np_arrays/indv_histos.npy",h1dict)

In [ ]:
string_selections = ["reco_NoCuts_"] #Reco Cuts. First two may not sum to 3rd

for s,xlim in zip(prefixes,xlims):
    for B_String,B in zip(B_Strings,B_Fields):
        fig = plt.figure(figsize=(18,12))
        plt.xticks([])
        plt.yticks([])
        plt.title(r"Full/Charged $\Delta\varphi$ Distributions",fontsize=40,y=1.04,verticalalignment="bottom")
        i=0
        for eta in range(N_eta):

            for p in range(N_mom):
                ax = fig.add_subplot(N_eta,N_divs,i+1)
                i+=1
                if eta==0:
                    p_range_string = r"$%1.1f < p < %1.1f$"%(mom_binning["edges"][p],mom_binning["edges"][p+1])
                    plt.text(0.5,1.18,p_range_string,ha="center",va="top",size=20,transform=ax.transAxes)
                if p==0:
                    eta_range_string = r"$%1.1f < \eta < %1.1f$"%(eta_binning["edges"][eta],eta_binning["edges"][eta+1])
                    #plt.text(0.5,1.12,eta_range_string,ha="center",va="top",size=15,alpha=0.7,transform=ax.transAxes)
                    plt.ylabel(eta_range_string,fontsize=16)
                for col in ["r","b","g"]:

                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    plt.errorbar(h1dict_full[s+"centers"],h1dict_full[identifier+"_vals"],yerr=h1dict_full[identifier+"_errors"],fmt=".",color=col,alpha=0.8)
                    
                    #aesthetics
                    plt.xlim(xlim)
                    plt.xticks(fontsize=8)
                    plt.yticks(fontsize=8)
                    mean = h1dict_full[identifier+"_avg"]
                    #plt.axvline(x=h1dict[identifier+"_avg"],color=col,linestyle="--",alpha=0.5)
                    #stddev = h1dict[identifier+"_sigma"]
                    #ax.axvspan(mean-stddev, mean+stddev, alpha=0.5, color=col)

                    plt.text(0.98,0.95,"B = %1.1f T"%(B),ha="right",va="top",size=15,alpha=0.7,transform=ax.transAxes)
        plt.tight_layout()
        plt.savefig(s+"_"+B_String+"_fulljet_dDeltaPhi_Overlays.pdf")

In [ ]:
for eta in range (N_eta):
    for p in range(N_mom):
        B_String = "3"
        s="eDelta_dph"
        identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
        id_full=("%s_B_%s_eta_%i_p_%i"%("efulljet_Delta_dph",B_String,eta,p))
        #print(h1dict[identifier+"_errors"][7],h1dict_full[id_full+"_errors"][7])

#print(h1dict.keys())

## Let's Fit in Python, because ROOT sucks

In [ ]:
#Set Fit Range for Single Gauss
fit_min_value = -0.005
fit_max_value = 0.005

arr = h1dict["eDelta_dph"+"centers"]
min_array = np.absolute(arr-fit_min_value)
max_array = np.absolute(arr-fit_max_value)
 
min_index = np.argmin(min_array)
max_index = np.argmin(max_array)+1

B_Fields = [1.4,3.0]
B_Strings = ["1p4","3"]
N_divs = N_mom
xlims=[[-0.04,0.04]]
prefixes = ["eDelta_dph"] #Types of Resolutions

## Double Gauss Fits

In [ ]:
string_selections = ["../../output/reco_NoCuts_"] #Reco Cuts. First two may not sum to 3rd
res_dict = {}

for s,xlim in zip(prefixes,xlims):
    for B_String,B in zip(B_Strings,B_Fields):
        fig = plt.figure(figsize=(18,12))
        plt.xticks([])
        plt.yticks([])
        plt.title(r"$\Delta\varphi$ Distributions (B = %1.1f T)"%(B),fontsize=40,y=1.04,verticalalignment="bottom")
        i=0
        for eta in range(N_eta):

            for p in range(N_mom):
                ax = fig.add_subplot(N_eta,N_divs,i+1)
                i+=1
                
                if eta==0:
                    p_range_string = r"$%1.1f < p < %1.1f$"%(mom_binning["edges"][p],mom_binning["edges"][p+1])
                    plt.text(0.5,1.18,p_range_string,ha="center",va="top",size=20,transform=ax.transAxes)
                if p==0:
                    eta_range_string = r"$%1.1f < \eta < %1.1f$"%(eta_binning["edges"][eta],eta_binning["edges"][eta+1])
                    plt.ylabel(eta_range_string,fontsize=16)
                
                for col in ["b","r","g"]:

                    identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                    plt.errorbar(h1dict[s+"centers"],h1dict[identifier+"_vals"],yerr=h1dict[identifier+"_errors"],fmt=".",color=col,alpha=0.8)
                    ydata = h1dict[identifier+"_vals"]
                    xdata=h1dict[s+"centers"]
                    popt, pcov = curve_fit(double_gauss, xdata, ydata, p0=[ydata.max(), xdata.mean(), xdata.std()/4,ydata.max()/5,xdata.mean(),xdata.std()])
                    plt.plot(xdata, double_gauss(xdata, *popt), 'r-', label='fit')
                    res_dict[identifier+"_dDeltaPhi_res"] = popt[5] #narrow gauss sigma
                    #aesthetics
                    plt.xlim(xlim)
                    plt.xticks(fontsize=8)
                    plt.yticks(fontsize=8)
                    
                    plt.text(0.98,0.95,"B = %1.1f T"%(B),ha="right",va="top",size=15,alpha=0.7,transform=ax.transAxes)
        plt.tight_layout()
        plt.savefig(s+"_"+B_String+"_dDeltaPhi_Overlays_Fits.pdf")
        print(popt[5])

### Weighted Average of Resolutions, to apply to $\Delta$ R Plots

In [ ]:
for s,xlim in zip(prefixes,xlims):
    for B_String,B in zip(B_Strings,B_Fields):
        weighted_avg = 0
        sum_N = 0
        for eta in range(N_eta):            
            for p in range(N_mom):
                identifier = ("%s_B_%s_eta_%i_p_%i"%(s,B_String,eta,p))
                #print(B,res_dict[identifier+"_dDeltaPhi_res"], h1dict[identifier+"_N"])
                weighted_avg += res_dict[identifier+"_dDeltaPhi_res"]*h1dict[identifier+"_N"]
                sum_N += h1dict[identifier+"_N"]
        print(B,weighted_avg/sum_N)
                
#print(res_dict.keys())

## e-Jet Correlations: Backward, Forward, and Central

#### From e_Jet_Distributions.cxx:283

In [ ]:
filename = "eta_bins_Histograms_Jet_Callibration_3.000000T.root"
filename_14T="eta_bins_Histograms_Jet_Callibration_1.400000T.root"

file = ROOT.TFile(filename)
file_14T = ROOT.TFile(filename_14T)

T_1p4 = {}
T_3 = {}

dPhi_bins,dPhi_centers,dPhi_widths = get_th1_binning_np(file,"dPhi_e_TrueJet_eta_0")


for et in range(3):
    TJ_dPhi, TJ_dPhi_errors = TH1_to_numpy_wErrors(file,"dPhi_e_TrueJet_eta_%i"%(et),False,False)
    RJ_dPhi, RJ_dPhi_errors = TH1_to_numpy_wErrors(file,"dPhi_e_RecoJet_eta_%i"%(et),False,False)

    Tesla14_RecodPhi,Tesla14_RecodPhi_errors = TH1_to_numpy_wErrors(file_14T,"dPhi_e_RecoJet_eta_%i"%(et),False,False)
    Tesla14_TruthdPhi,Tesla14_TruthdPhi_errors = TH1_to_numpy_wErrors(file_14T,"dPhi_e_TrueJet_eta_%i"%(et),False,False)

    all_TJ_dPhi, all_TJ_dPhi_errors = TH1_to_numpy_wErrors(file,"all_dPhi_e_TrueJet_eta_%i"%(et),False,False)

    TJ_dPhi = TJ_dPhi/dPhi_widths
    TJ_dPhi_errors = TJ_dPhi_errors/dPhi_widths
    RJ_dPhi = RJ_dPhi/dPhi_widths
    RJ_dPhi_errors = RJ_dPhi_errors/dPhi_widths

    Tesla14_RecodPhi = Tesla14_RecodPhi/dPhi_widths
    Tesla14_RecodPhi_errors = Tesla14_RecodPhi_errors/dPhi_widths 
    Tesla14_TruthdPhi = Tesla14_TruthdPhi/dPhi_widths
    Tesla14_TruthdPhi_errors = Tesla14_TruthdPhi_errors/dPhi_widths

    all_TJ_dPhi = all_TJ_dPhi/dPhi_widths
    all_TJ_dPhi_errors = all_TJ_dPhi_errors/dPhi_widths
    
    T_1p4["truth_vals_eta_%i"%(et)] = Tesla14_TruthdPhi
    T_1p4["truth_err_eta_%i"%(et)] = Tesla14_TruthdPhi_errors
    T_1p4["reco_vals_eta_%i"%(et)] = Tesla14_RecodPhi
    T_1p4["reco_err_eta_%i"%(et)] = Tesla14_TruthdPhi_errors
    
    T_3["all_truth_vals_eta_%i"%(et)] = all_TJ_dPhi
    T_3["all_truth_err_eta_%i"%(et)] = all_TJ_dPhi_errors
    
    T_3["truth_vals_eta_%i"%(et)] = TJ_dPhi
    T_3["truth_err_eta_%i"%(et)] = TJ_dPhi_errors
    T_3["reco_vals_eta_%i"%(et)] = RJ_dPhi
    T_3["reco_err_eta_%i"%(et)] = RJ_dPhi_errors
   
    

In [ ]:
cool = get_colors(plt.cm.winter,4,False)

for et in range(3):
    
    fig = plt.figure(figsize=(14,12))

#3.0T Field
    plt.errorbar(dPhi_centers,T_3["truth_vals_eta_%i"%(et)],yerr=T_3["truth_err_eta_%i"%(et)],
                         fmt='-',color=cool[0],fillstyle='none',markersize=7,linewidth=3,
                         label=r"$|\Delta\varphi_\mathrm{Truth}|~B = 3.0$ T" )

    plt.errorbar(dPhi_centers,T_3["reco_vals_eta_%i"%(et)],yerr=T_3["reco_err_eta_%i"%(et)],
                         fmt='--',color=cool[1],linewidth=3,alpha=0.8,
                         label=r"$|\Delta\varphi_\mathrm{Reco}|~B = 3.0$ T" )

#1.4T Field
    plt.errorbar(dPhi_centers,T_1p4["truth_vals_eta_%i"%(et)],yerr=T_1p4["truth_err_eta_%i"%(et)],
                         fmt='-',color=cool[2],fillstyle='none',markersize=7,linewidth=3,
                         label=r"$|\Delta\varphi_\mathrm{Truth}|~B = 1.4$ T" )
                         #label=r"$|\varphi_{truth}^{jet} - \varphi^{e}-\pi|$")

    plt.errorbar(dPhi_centers,T_1p4["reco_vals_eta_%i"%(et)],yerr=T_1p4["reco_err_eta_%i"%(et)],
                         fmt='--',color=cool[3],linewidth=3,alpha=0.8,
                         label=r"$|\Delta\varphi_\mathrm{Reco}|~B = 1.4$ T")
                         #label=r"$|\varphi_{reco}^{jet} - \varphi^{e}-\pi|$")

    plt.tight_layout()
    plt.xlim(0,.5)
    plt.ylabel("Raw Counts $\mathrm{d}N/\mathrm{d}\Delta\varphi$",fontsize=25,y=0.5)
    plt.xlabel(r"$|\Delta\varphi|=|\varphi^{jet} - \varphi^{e}-\pi|$",fontsize=25,x=0.5)
    plt.tick_params(which='both',direction='in',right=True,top=True,bottom=True,length=10,labelsize=20)
    plt.legend(fontsize=25,loc='upper right')
    plt.savefig("azimuthal_correlations_eta_bins.pdf")